# コンテナを利用して Lambda 関数を作成する
このスクリプトを実行するための IAM の権限や、AWS configure とかは事前に要設定

In [1]:
import boto3, datetime, subprocess, json
from time import sleep

In [2]:
# 様々な名前をユニークにするため、現在時刻のyyyymmddhhmmssを利用する
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d%H%M%S')
image = 'lambda-container-yolov5'
tag = ':latest'
repository_name = f'{image}-{now}'

## ECR リポジトリを boto3 で作成

In [3]:
%%time
# ECR のリポジトリを作成する
ecr = boto3.client('ecr')
response = ecr.create_repository(
    repositoryName=repository_name,
    imageScanningConfiguration={'scanOnPush': True},
)

CPU times: user 44.8 ms, sys: 4.08 ms, total: 48.9 ms
Wall time: 76.9 ms


In [4]:
# 必要な情報を抜き取っておく
uri = response['repository']['repositoryUri']
account_id = response['repository']['registryId']
region = uri.split('.')[3]
domain = uri.split('/')[0]

In [5]:
response

{'repository': {'repositoryArn': 'arn:aws:ecr:us-east-2:588279871613:repository/lambda-container-yolov5-20210322234155',
  'registryId': '588279871613',
  'repositoryName': 'lambda-container-yolov5-20210322234155',
  'repositoryUri': '588279871613.dkr.ecr.us-east-2.amazonaws.com/lambda-container-yolov5-20210322234155',
  'createdAt': datetime.datetime(2021, 3, 22, 14, 41, 57, tzinfo=tzlocal()),
  'imageTagMutability': 'MUTABLE',
  'imageScanningConfiguration': {'scanOnPush': True},
  'encryptionConfiguration': {'encryptionType': 'AES256'}},
 'ResponseMetadata': {'RequestId': 'df5465a1-df32-4714-9d86-1a075a6612d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'df5465a1-df32-4714-9d86-1a075a6612d1',
   'date': 'Mon, 22 Mar 2021 14:41:56 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '467'},
  'RetryAttempts': 0}}

## ECR にコンテナイメージをプッシュ

In [6]:
%%time

# build からプッシュまで

# build 済なら実行不要
# !docker build -t {image} .

# タグ付与
!sudo docker tag {image}{tag} {uri}{tag}

# ECR にログイン
! aws ecr get-login-password | sudo docker login --username AWS --password-stdin {domain}

# push
!sudo docker push {uri}{tag}

WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [588279871613.dkr.ecr.us-east-2.amazonaws.com/lambda-container-yolov5-20210322234155]

fe4691f8: Preparing 
40d0b654: Preparing 
b56a7423: Preparing 
451a3084: Preparing 
0f1abfed: Preparing 
9049197e: Preparing 
1d87ffd3: Preparing 
1c625349: Preparing 
de1526b2: Preparing 
fc3c462b: Preparing 
943798ba: Preparing 
f826a39a: Preparing 
1e0b1647: Preparing 
7ce9393d: Preparing 


f1abfed: Pushing  1.049GB/2.976GBPushing  124.1MB/2.976GBPushing  227.8MB/934.3MBPushing   24.9MB/28.42MBPushing  332.5MB/934.3MBPushing  239.2MB/2.976GBPushing  922.4MB/934.3MB

f1abfed: Pushed   2.988GB/2.976GBPushing  1.055GB/2.976GBPushing  1.313GB/2.976GBPushing  2.254GB/2.976GBPushing  2.341GB/2.976GBPushing  2.491GB/2.976GBPushing  2.773GB/2.976GBPushing  2.977GBlatest: digest: sha256:4d2179dadf9e3815948daa4fd8d071be0c81ce7b0d6458182eecccce4c9e551f size: 3476
CPU times: user 2.76 s, sys: 625 ms, total: 3.39 s
Wall time: 2min 44s


In [7]:
# 必要な情報を抜き取っておく
res = ecr.describe_images(
    repositoryName = repository_name
)
image_digest = res['imageDetails'][0]['imageDigest']

## Lambda を実行するロールを設定
1. ロール作成
2. ポリシー作成
3. ポリシーをロールにアタッチ

In [8]:
iam = boto3.client('iam')
function_name = f'{image}-function-{now}'
doc = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Action': 'sts:AssumeRole',
            'Principal': {
                'Service': 'lambda.amazonaws.com'
                
            },
            'Effect': 'Allow',
            'Sid': ''
            
        }
    ]
}

# ロール作成
role_name = f'{image}-role-{now}'
res = iam.create_role(
    Path = '/service-role/',
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(doc),
    Description=f'exec role',
    MaxSessionDuration=3600*12
)
role_arn = res['Role']['Arn']

doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": f"arn:aws:logs:{account_id}:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                f"arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{function_name}:*"
            ]
        }
    ]
}

# ポリシー作成
poicy_name = f'{image}-policy-{now}'
res = iam.create_policy(
    PolicyName=poicy_name,
    PolicyDocument=json.dumps(doc),
)
policy_arn = res['Policy']['Arn']

# 作成したポリシーをロールにアタッチ
res = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

# IAM の反映をしばし待つ
sleep(20)

## Lambda 関数を作成する
IAM の反映が間に合わない場合があるので、 sleep を入れているが、十分に時間が経っている場合は不要

In [9]:
%%time
# lambda function を Docker で作成したイメージから作成する
lambda_client = boto3.client('lambda')
res = lambda_client.create_function(
    FunctionName=function_name,
    Role=role_arn,
    Code={
        'ImageUri': f'{uri}@{image_digest}'
    },
    Description='input-> b64img, output -> label(json), yolov5 detect',
    Timeout=60*15,
    MemorySize=4096,#1024,
    Publish=True,
    PackageType='Image',
)
# 作成が完了するまで待つ
while True:
    res = lambda_client.get_function(FunctionName=function_name)
    try:
        if res['Configuration']['StateReasonCode']=='Creating':
            print('.',end='')
            sleep(1)
    except:
        if res['Configuration']['LastUpdateStatus']=='Successful':
            print('!')
            break
        else:
            print('?')
            break


....................................................................................................!
CPU times: user 404 ms, sys: 8.66 ms, total: 413 ms
Wall time: 1min 43s


## 推論してみる

In [10]:
import base64
import requests
import cv2
import numpy as np
from matplotlib import pyplot as plt
input_file = './yolov5/data/images/20.jpg'
data = {}
with open(input_file,'rb') as f:
    data['img']= base64.b64encode(f.read()).decode('utf-8')

In [13]:
%%time
res = lambda_client.invoke(
    FunctionName=function_name,
    Payload=json.dumps(data)
)

CPU times: user 9.29 ms, sys: 0 ns, total: 9.29 ms
Wall time: 20.4 s


In [14]:
label = json.loads(res['Payload'].read())['label']
print(label)

['イヌ', 'ネコ', 'クマ', 'UR', 'ウサギ']


In [ ]:
#img_b64 = json.loads(res['Payload'].read())['img']
#img_bin = base64.b64decode(img_b64)
#img_array = np.frombuffer(img_bin,dtype=np.uint8)
#img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
#plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

## お片付け

In [ ]:
lambda_client.delete_function(
    FunctionName=function_name
)

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName=role_name,
)

In [ ]:
ecr.batch_delete_image(
    repositoryName=repository_name,
    imageIds=[{
        'imageDigest':image_digest
    }]
)

In [ ]:
ecr.delete_repository(
    repositoryName=repository_name,
)